# Pipe 공정 순서배열에 따른 소요시간 최적화 방법론

엑셀 데이터를 기반으로 제품 공정의 스케줄 최적화를 위한 Python 알고리즘을 개발하려고 합니다. 엑셀 데이터는 각 제품의 공정 단계(1단계부터 6단계까지)와 해당 단계의 평균 작업 시간을 포함하고 있습니다.

데이터 구조:

제품그룹: 제품 그룹을 나타내는 고유 식별자.
step1 평균 작업 시간(분) ~ step6 평균 작업 시간(분): 각 공정 단계의 평균 작업 시간.

제품개수: 각 제품 그룹별로 생산해야 할 제품의 수.

공정 조건:
각 제품은 1단계부터 6단계까지 순차적으로 공정을 진행해야 합니다.
각 공정 단계는 병렬 처리가 가능하지만, 다음과 같은 기계(호기) 제한이 있습니다:
1단계, 2단계, 5단계, 6단계: 각 단계당 1호기.
3단계: 3호기.
4단계: 4호기.
각 호기는 동시에 하나의 제품만을 처리할 수 있습니다.
제품의 공정 순서를 최적화하여 총 작업 소요 시간을 최소화하고, 대기 시간을 줄이고자 합니다.

예시 상황:
일부 제품은 1~5단계는 빠르게 완료되지만 6단계가 오래 걸릴 수 있습니다. 이러한 제품을 우선적으로 처리하여 전체 대기 시간을 줄이고자 합니다.
예를 들어, 제품 15가 3단계를 빠르게 완료했지만 제품 23이 4단계에서 오래 걸려 다음 단계로 넘어가는 데 지연이 발생할 수 있습니다. 이러한 대기 시간을 최소화하기 위해 제품의 공정 시작 순서를 최적화해야 합니다.

목표:
모든 제품의 공정을 완료했을 때의 총 작업 소요 시간과 최적화된 제품 순서를 도출하는 Python 코드를 작성합니다.

In [ ]:
# Pipe 공정의 자료구조는 모두 queue 구조를 따릅니다.

## 데이터 확인
1.   Raw 데이터
2.   GT값 (휴리스틱라벨링)
3.   정규분포에 따른 데이터 필터링
4.   our 데이터 전처리 시스템

*   같은 파이프 종류를 그룹별로 함께 작업하는 경우
*   전체 파이프에 대해 하나하나 따로 작업하는 경우



In [7]:
import pandas as pd
#1
#file_path = 'pipe_final.xlsx'
#2
#file_path = 'heuristic_data.xlsx'
#3
#file_path = 'filter_pipe_final_241007.xlsx'
#4
#file_path = 'updated_total.xlsx'
#5
file_path = 'updated_total_filter.xlsx'

## OR tools 가장 기본

In [2]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.2

In [ ]:
"""
# 4,5 데이터에 해당함
for idx, row in df.iterrows():
    product_group = row['제품군']
    processing_times = {
        'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
        'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
        'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
        'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
        'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
        'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
    }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })
--------------------------------------------------------------------------------------
# 1,2,3 데이터에 해당함
    for idx, row in df.iterrows():
        product_group = row['제품번호']
        processing_times_dict = {
            'step1': int(round(row['step1 작업 시간(분)'] * SCALE)),
            'step2': int(round(row['step2 작업 시간(분)'] * SCALE)),
            'step3': int(round(row['step3 작업 시간(분)'] * SCALE)),
            'step4': int(round(row['step4 작업 시간(분)'] * SCALE)),
            'step5': int(round(row['step5 작업 시간(분)'] * SCALE)),
            'step6': int(round(row['step6 작업 시간(분)'] * SCALE)),
        }
        units.append({
            'unit_id': f"{product_group}",
            'product_group': product_group,
            'processing_times': processing_times_dict,
        })


"""


# 1번경우를 고려한 최적화 방법론

In [8]:
import pandas as pd
from ortools.sat.python import cp_model

# 엑셀 데이터를 읽어옵니다.
df = pd.read_excel(file_path)

# NaN 값이 있는 행을 제거합니다.
df.dropna(inplace=True)

# 제품 데이터를 단위 제품으로 만듭니다.
units = []
# **시간 단위를 정수로 변환하기 위한 스케일링 팩터를 정의합니다.**
SCALE = 1000  # 소수점 이하 3자리까지 표현


for idx, row in df.iterrows():
    product_group = row['제품군']
    processing_times = {
        'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
        'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
        'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
        'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
        'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
        'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
    }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })

# 스케줄링 문제를 정의합니다.
model = cp_model.CpModel()

steps = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
machines_per_step = {
    'step1': 1,
    'step2': 1,
    'step3': 3,
    'step4': 4,
    'step5': 1,
    'step6': 1,
}

# 각 단위 제품과 단계에 대한 변수들을 정의합니다.
horizon = sum([u['processing_times'][s] for u in units for s in steps])
unit_step_intervals = {}

for unit in units:
    unit_id = unit['unit_id']
    processing_times = unit['processing_times']
    for step in steps:
        duration = int(processing_times[step])
        start_var = model.NewIntVar(0, horizon, f'start_{unit_id}_{step}')
        end_var = model.NewIntVar(0, horizon, f'end_{unit_id}_{step}')
        interval_var = model.NewIntervalVar(start_var, duration, end_var, f'interval_{unit_id}_{step}')
        unit_step_intervals[(unit_id, step)] = interval_var

# 단계별 선행 조건을 추가합니다.
for unit in units:
    unit_id = unit['unit_id']
    for s in range(len(steps) - 1):
        step_current = steps[s]
        step_next = steps[s + 1]
        end_current = unit_step_intervals[(unit_id, step_current)].EndExpr()
        start_next = unit_step_intervals[(unit_id, step_next)].StartExpr()
        model.Add(end_current <= start_next)

# 각 단계의 기계 용량 제약 조건을 추가합니다.
for step in steps:
    intervals = []
    demands = []
    for unit in units:
        unit_id = unit['unit_id']
        intervals.append(unit_step_intervals[(unit_id, step)])
        demands.append(1)
    machine_capacity = machines_per_step[step]
    model.AddCumulative(intervals, demands, machine_capacity)

# 총 작업 시간을 최소화하는 목표를 설정합니다.
makespan = model.NewIntVar(0, horizon, 'makespan')
end_times = []
for unit in units:
    unit_id = unit['unit_id']
    end_times.append(unit_step_intervals[(unit_id, 'step6')].EndExpr())
model.AddMaxEquality(makespan, end_times)
model.Minimize(makespan)

# 모델을 해결합니다.
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

# 결과를 출력합니다.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"총 작업 시간: {solver.ObjectiveValue()} 분")
    schedule = []
    for unit in units:
        unit_id = unit['unit_id']
        unit_schedule = {'unit_id': unit_id, 'steps': {}}
        for step in steps:
            start = solver.Value(unit_step_intervals[(unit_id, step)].StartExpr())
            end = solver.Value(unit_step_intervals[(unit_id, step)].EndExpr())
            unit_schedule['steps'][step] = {'start': start, 'end': end}
        schedule.append(unit_schedule)
    schedule.sort(key=lambda x: x['steps']['step1']['start'])
    for unit_schedule in schedule:
        unit_id = unit_schedule['unit_id']
        print(f"제품 그룹: {unit_id}")
        for step in steps:
            start = unit_schedule['steps'][step]['start']
            end = unit_schedule['steps'][step]['end']
            print(f"  {step}: {start} 분에 시작, {end} 분에 종료")
else:
    print("해결 가능한 솔루션을 찾지 못했습니다.")


총 작업 시간: 1932700.0 분
제품 그룹: 23D016-05
  step1: 0 분에 시작, 8100 분에 종료
  step2: 8100 분에 시작, 17800 분에 종료
  step3: 17800 분에 시작, 31200 분에 종료
  step4: 31200 분에 시작, 40300 분에 종료
  step5: 41800 분에 시작, 50400 분에 종료
  step6: 55200 분에 시작, 68100 분에 종료
제품 그룹: 23D017-32
  step1: 8100 분에 시작, 19200 분에 종료
  step2: 19200 분에 시작, 25200 분에 종료
  step3: 25200 분에 시작, 28600 분에 종료
  step4: 28600 분에 시작, 29600 분에 종료
  step5: 29600 분에 시작, 41800 분에 종료
  step6: 41800 분에 시작, 55200 분에 종료
제품 그룹: 23E004-02
  step1: 19200 분에 시작, 27100 분에 종료
  step2: 27100 분에 시작, 40400 분에 종료
  step3: 40400 분에 시작, 51700 분에 종료
  step4: 51700 분에 시작, 58200 분에 종료
  step5: 58200 분에 시작, 64500 분에 종료
  step6: 68100 분에 시작, 82400 분에 종료
제품 그룹: 23E009-03
  step1: 27100 분에 시작, 37200 분에 종료
  step2: 40400 분에 시작, 50400 분에 종료
  step3: 50400 분에 시작, 58900 분에 종료
  step4: 58900 분에 시작, 71200 분에 종료
  step5: 71200 분에 시작, 80100 분에 종료
  step6: 82400 분에 시작, 94000 분에 종료
제품 그룹: 23D017-29
  step1: 37200 분에 시작, 48300 분에 종료
  step2: 50400 분에 시작, 61500 분에 종료
  step3: 61500 분에

# 2번 경우를 고려한 최적화방법

In [13]:
import pandas as pd
from ortools.sat.python import cp_model

# 엑셀 데이터를 읽어옵니다.
df = pd.read_excel(file_path)

# NaN 값이 있는 행을 제거합니다.
df.dropna(inplace=True)

# 시간 단위를 정수로 변환하기 위한 스케일링 팩터를 정의합니다.
SCALE = 1000  # 소수점 이하 3자리까지 표현

# 제품 데이터를 단위 제품으로 만듭니다.
units = []

for idx, row in df.iterrows():
    processing_times = {
        'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
        'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
        'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
        'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
        'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
        'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
    }
    units.append({
        'unit_id': f"unit_{idx}",
        'processing_times': processing_times,
    })

# 스케줄링 문제를 정의합니다.
model = cp_model.CpModel()

steps = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
machines_per_step = {
    'step1': 1,
    'step2': 1,
    'step3': 3,
    'step4': 4,
    'step5': 1,
    'step6': 1,
}

# 각 단위 제품과 단계에 대한 변수들을 정의합니다.
horizon = sum([u['processing_times'][s] for u in units for s in steps])
unit_step_intervals = {}

for unit in units:
    unit_id = unit['unit_id']
    processing_times = unit['processing_times']
    for step in steps:
        duration = processing_times[step]
        start_var = model.NewIntVar(0, horizon, f'start_{unit_id}_{step}')
        end_var = model.NewIntVar(0, horizon, f'end_{unit_id}_{step}')
        interval_var = model.NewIntervalVar(start_var, duration, end_var, f'interval_{unit_id}_{step}')
        unit_step_intervals[(unit_id, step)] = interval_var

# 단계별 선행 조건을 추가합니다.
for unit in units:
    unit_id = unit['unit_id']
    for s in range(len(steps) - 1):
        step_current = steps[s]
        step_next = steps[s + 1]
        end_current = unit_step_intervals[(unit_id, step_current)].EndExpr()
        start_next = unit_step_intervals[(unit_id, step_next)].StartExpr()
        model.Add(end_current <= start_next)

# 각 단계의 기계 용량 제약 조건을 추가합니다.
for step in steps:
    intervals = []
    for unit in units:
        unit_id = unit['unit_id']
        intervals.append(unit_step_intervals[(unit_id, step)])
    machine_capacity = machines_per_step[step]
    model.AddNoOverlap(intervals)

    if machine_capacity > 1:
        # 기계 용량이 1보다 큰 경우, Cumulative 제약 조건을 사용합니다.
        demands = [1] * len(intervals)
        model.AddCumulative(intervals, demands, machine_capacity)

# 총 작업 시간을 최소화하는 목표를 설정합니다.
makespan = model.NewIntVar(0, horizon, 'makespan')
end_times = [unit_step_intervals[(unit['unit_id'], 'step6')].EndExpr() for unit in units]
model.AddMaxEquality(makespan, end_times)
model.Minimize(makespan)

# 모델을 해결합니다.
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

# 결과를 출력합니다.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    total_time = solver.Value(makespan) / SCALE  # 원래 시간 단위로 환산
    print(f"총 작업 시간: {total_time} 분")
    schedule = []
    for unit in units:
        unit_id = unit['unit_id']
        unit_schedule = {'unit_id': unit_id, 'steps': {}}
        for step in steps:
            start = solver.Value(unit_step_intervals[(unit_id, step)].StartExpr()) / SCALE
            end = solver.Value(unit_step_intervals[(unit_id, step)].EndExpr()) / SCALE
            unit_schedule['steps'][step] = {'start': start, 'end': end}
        schedule.append(unit_schedule)
    schedule.sort(key=lambda x: x['steps']['step1']['start'])
    for unit_schedule in schedule:
        unit_id = unit_schedule['unit_id']
        print(f"제품: {unit_id}")
        for step in steps:
            start = unit_schedule['steps'][step]['start']
            end = unit_schedule['steps'][step]['end']
            print(f"  {step}: {start:.3f} 분에 시작, {end:.3f} 분에 종료")
else:
    print("해결 가능한 솔루션을 찾지 못했습니다.")


총 작업 시간: 1928.9 분
제품: unit_85
  step1: 0.000 분에 시작, 8.400 분에 종료
  step2: 8.400 분에 시작, 18.500 분에 종료
  step3: 18.500 분에 시작, 22.900 분에 종료
  step4: 22.900 분에 시작, 29.700 분에 종료
  step5: 29.700 분에 시작, 37.800 분에 종료
  step6: 37.800 분에 시작, 54.000 분에 종료
제품: unit_117
  step1: 8.400 분에 시작, 17.000 분에 종료
  step2: 18.500 분에 시작, 30.300 분에 종료
  step3: 30.300 분에 시작, 37.700 분에 종료
  step4: 37.700 분에 시작, 46.000 분에 종료
  step5: 46.000 분에 시작, 52.300 분에 종료
  step6: 54.000 분에 시작, 508.800 분에 종료
제품: unit_59
  step1: 20.600 분에 시작, 29.400 분에 종료
  step2: 31.900 분에 시작, 41.900 분에 종료
  step3: 42.300 분에 시작, 54.700 분에 종료
  step4: 54.700 분에 시작, 63.400 분에 종료
  step5: 70.700 분에 시작, 93.500 분에 종료
  step6: 972.300 분에 시작, 992.700 분에 종료
제품: unit_4
  step1: 29.400 분에 시작, 40.100 분에 종료
  step2: 41.900 분에 시작, 53.400 분에 종료
  step3: 54.700 분에 시작, 65.100 분에 종료
  step4: 65.100 분에 시작, 75.400 분에 종료
  step5: 93.500 분에 시작, 110.000 분에 종료
  step6: 653.800 분에 시작, 675.600 분에 종료
제품: unit_15
  step1: 40.100 분에 시작, 54.500 분에 종료
  step2: 54.500 분에 시

#Pipe의 공정 순서 배열 엑셀 데이터를 바탕으로 공정 전체 소요시간을 계산하는 코드

In [14]:
import pandas as pd
from ortools.sat.python import cp_model

# 엑셀 데이터를 읽어옵니다.
df = pd.read_excel(file_path)

# NaN 값이 있는 행을 제거합니다.
df.dropna(inplace=True)

# 제품 데이터를 단위 제품으로 만듭니다.
units = []
# **시간 단위를 정수로 변환하기 위한 스케일링 팩터를 정의합니다.**
SCALE = 1000  # 소수점 이하 3자리까지 표현


for idx, row in df.iterrows():
    product_group = row['제품군']
    processing_times = {
        'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
        'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
        'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
        'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
        'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
        'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
    }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })

# 스케줄링 문제를 정의합니다.
model = cp_model.CpModel()

steps = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
machines_per_step = {
    'step1': 1,
    'step2': 1,
    'step3': 3,
    'step4': 4,
    'step5': 1,
    'step6': 1,
}

# 각 단위 제품과 단계에 대한 변수들을 정의합니다.
horizon = sum([u['processing_times'][s] for u in units for s in steps])
unit_step_intervals = {}

for unit in units:
    unit_id = unit['unit_id']
    processing_times = unit['processing_times']
    for step in steps:
        duration = int(processing_times[step])
        start_var = model.NewIntVar(0, horizon, f'start_{unit_id}_{step}')
        end_var = model.NewIntVar(0, horizon, f'end_{unit_id}_{step}')
        interval_var = model.NewIntervalVar(start_var, duration, end_var, f'interval_{unit_id}_{step}')
        unit_step_intervals[(unit_id, step)] = interval_var

# 단계별 선행 조건을 추가합니다.
for unit in units:
    unit_id = unit['unit_id']
    for s in range(len(steps) - 1):
        step_current = steps[s]
        step_next = steps[s + 1]
        end_current = unit_step_intervals[(unit_id, step_current)].EndExpr()
        start_next = unit_step_intervals[(unit_id, step_next)].StartExpr()
        model.Add(end_current <= start_next)

# 각 단계의 기계 용량 제약 조건을 추가합니다.
for step in steps:
    intervals = []
    demands = []
    for unit in units:
        unit_id = unit['unit_id']
        intervals.append(unit_step_intervals[(unit_id, step)])
        demands.append(1)
    machine_capacity = machines_per_step[step]
    model.AddCumulative(intervals, demands, machine_capacity)

# 총 작업 시간을 최소화하는 목표를 설정합니다.
makespan = model.NewIntVar(0, horizon, 'makespan')
end_times = []
for unit in units:
    unit_id = unit['unit_id']
    end_times.append(unit_step_intervals[(unit_id, 'step6')].EndExpr())
model.AddMaxEquality(makespan, end_times)
model.Minimize(makespan)

# 모델을 해결합니다.
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

# 결과를 출력합니다.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"총 작업 시간: {solver.ObjectiveValue()} 분")
    schedule = []
    for unit in units:
        unit_id = unit['unit_id']
        unit_schedule = {'unit_id': unit_id, 'steps': {}}
        for step in steps:
            start = solver.Value(unit_step_intervals[(unit_id, step)].StartExpr())
            end = solver.Value(unit_step_intervals[(unit_id, step)].EndExpr())
            unit_schedule['steps'][step] = {'start': start, 'end': end}
        schedule.append(unit_schedule)
    schedule.sort(key=lambda x: x['steps']['step1']['start'])
    for unit_schedule in schedule:
        unit_id = unit_schedule['unit_id']
        print(f"제품 그룹: {unit_id}")
        for step in steps:
            start = unit_schedule['steps'][step]['start']
            end = unit_schedule['steps'][step]['end']
            print(f"  {step}: {start} 분에 시작, {end} 분에 종료")
else:
    print("해결 가능한 솔루션을 찾지 못했습니다.")


총 작업 시간: 1932700.0 분
제품 그룹: 23D016-05
  step1: 0 분에 시작, 8100 분에 종료
  step2: 8100 분에 시작, 17800 분에 종료
  step3: 17800 분에 시작, 31200 분에 종료
  step4: 31200 분에 시작, 40300 분에 종료
  step5: 41800 분에 시작, 50400 분에 종료
  step6: 55200 분에 시작, 68100 분에 종료
제품 그룹: 23D017-32
  step1: 8100 분에 시작, 19200 분에 종료
  step2: 19200 분에 시작, 25200 분에 종료
  step3: 25200 분에 시작, 28600 분에 종료
  step4: 28600 분에 시작, 29600 분에 종료
  step5: 29600 분에 시작, 41800 분에 종료
  step6: 41800 분에 시작, 55200 분에 종료
제품 그룹: 23E004-02
  step1: 19200 분에 시작, 27100 분에 종료
  step2: 27100 분에 시작, 40400 분에 종료
  step3: 40400 분에 시작, 51700 분에 종료
  step4: 51700 분에 시작, 58200 분에 종료
  step5: 58200 분에 시작, 64500 분에 종료
  step6: 68100 분에 시작, 82400 분에 종료
제품 그룹: 23E009-03
  step1: 27100 분에 시작, 37200 분에 종료
  step2: 40400 분에 시작, 50400 분에 종료
  step3: 50400 분에 시작, 58900 분에 종료
  step4: 58900 분에 시작, 71200 분에 종료
  step5: 71200 분에 시작, 80100 분에 종료
  step6: 82400 분에 시작, 94000 분에 종료
제품 그룹: 23D017-29
  step1: 37200 분에 시작, 48300 분에 종료
  step2: 50400 분에 시작, 61500 분에 종료
  step3: 61500 분에

In [ ]:
# 코드스니펫
"""
'step1 작업 시간(분)', 'step2 작업 시간(분)',
         'step3 작업 시간(분)', 'step4 작업 시간(분)',
         'step5 작업 시간(분)', 'step6 작업 시간(분)'

for idx, row in df.iterrows():
    product_group = row['제품군']
    processing_times = {
        'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
        'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
        'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
        'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
        'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
        'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
    }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })